In [1]:
#-*-coding:utf-8-*-
import util
import tensorflow as tf
MNIST_CLASSIFIER_FROZEN_GRAPH = './classify_mnist_graph_def.pb'
INPUT_TENSOR = 'inputs:0'
OUTPUT_TENSOR = 'logits:0'
def EuclideanDistances(A, B):

    BT = B.transpose()
    # vecProd = A * BT
    vecProd = np.dot(A, BT)
    # print(vecProd)
    SqA = A ** 2
    # print(SqA)
    sumSqA = np.matrix(np.sum(SqA, axis=1))
    sumSqAEx = np.tile(sumSqA.transpose(), (1, vecProd.shape[1]))
    # print(sumSqAEx)

    SqB = B ** 2
    sumSqB = np.sum(SqB, axis=1)
    sumSqBEx = np.tile(sumSqB, (vecProd.shape[0], 1))
    SqED = sumSqBEx + sumSqAEx - 2 * vecProd
    SqED[SqED < 0] = 0.0
    ED = np.sqrt(SqED)
    return np.divide(ED.sum(),ED.shape[0]*ED.shape[1])
def cal_distance_image_real(images,labels):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)
def cal_distance_image_fake(images):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    labels = tf.Session().run(tf.argmax(y_logits, 1))
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)

    

In [4]:
import os
if not os.path.isdir('saved_models_{}'.format('cgan')):
    os.mkdir('saved_models_{}'.format('cgan'))
f = open('saved_models_{}/log_collapse1.txt'.format('cgan'), mode='w')


from keras.datasets import fashion_mnist,mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt

import numpy as np
import os
class CGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        img = self.generator([noise, label])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        valid = self.discriminator([img, label])

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model([noise, label], valid)
        self.combined.compile(loss=['binary_crossentropy'],
            optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(512 * 7 * 7, input_dim=self.latent_dim))
        model.add(Reshape((7,7, 512)))
        model.add(Activation("relu"))

        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(256, kernel_size=3, padding="same"))
        model.add(Activation("relu"))

        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3,strides=1,padding="same"))
        model.add(Activation("relu"))

        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3,strides=1, padding="same"))
        model.add(Activation("relu"))

        model.add(Conv2D(self.channels, strides=2,kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, self.latent_dim)(label))

        model_input = multiply([noise, label_embedding])
        img = model(model_input)

        return Model([noise, label], img)
    def build_discriminator(self):
        model = Sequential()
        model.add(Dense(self.img_rows*self.img_cols*self.channels, input_dim=np.prod(self.img_shape)))
        model.add(Reshape((self.img_rows,self.img_cols,self.channels)))

        model.add(Conv2D(64, kernel_size=3, strides=2,padding="same"))
        model.add(LeakyReLU(alpha=0.2))

        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))

        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(256, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))

        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(512, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()
        img = Input(shape=self.img_shape)
        label = Input(shape=(1,), dtype='int32')

        label_embedding = Flatten()(Embedding(self.num_classes, np.prod(self.img_shape))(label))
        flat_img = Flatten()(img)

        model_input = multiply([flat_img, label_embedding])

        validity = model(model_input)

        return Model([img, label], validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (X_test, y_test) = mnist.load_data()

        # Configure input
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5
        # X_test = X_test / 127.5 - 1.
        X_test = np.expand_dims(X_test, axis=3)

        y_train = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                imgs = X_train[index * batch_size:(index + 1) * batch_size]
                labels = y_train[index * batch_size:(index + 1) * batch_size]

                # Sample noise as generator input
                noise = np.random.normal(0, 1, (batch_size, 100))

                # Generate a half batch of new images
                gen_imgs = self.generator.predict([noise, labels])

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch([imgs, labels], valid)
                d_loss_fake = self.discriminator.train_on_batch([gen_imgs, labels], fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                # Condition on labels
                sampled_labels = np.random.randint(0, 10, batch_size).reshape(-1, 1)

                # Train the generator
                g_loss = self.combined.train_on_batch([noise, sampled_labels], valid)

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch,global_step, d_loss[0], 100 * d_loss[1], g_loss))

                sampleSize = 5000
                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    s = self.sample_images(global_step, X_test,y_test, sampleSize)
    def sample_images(self, epoch,x_test,y_test,sample_num):
        r, c = sample_num//10, 10
        noise = np.random.normal(0, 1, (r * c, 100))
        sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise,sampled_labels])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        labels = np.squeeze(y_test[:sample_num])
        labels = np.squeeze(labels)
        dis_real = cal_distance_image_real(x_test[:sample_num], labels)
        dis_fake = cal_distance_image_fake(gen_imgs)
        dis_cha = dis_real - dis_fake
        print('##############')
        # print(dis_real)
        # print(dis_fake)
        print(dis_cha)
        print('##########')
        f.writelines('\n')
        f.writelines('epoch:' + str(epoch))
        f.writelines('\n')
        f.writelines('紧度')
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in dis_cha)
        f.writelines('\n')
        



if __name__ == '__main__':
    cgan = CGAN()
    cgan.train(epochs=30, batch_size=64, sample_interval=200)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 784)               615440    
_________________________________________________________________
reshape_5 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 14, 14, 64)        640       
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 14, 14, 64)        256       
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 7, 7, 128)         73856     
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 7, 7, 128)         0         
__________

epoch:0 step:50 [D loss: 0.104004, acc.: 97.66%] [G loss: 4.518100]
epoch:0 step:51 [D loss: 0.082686, acc.: 100.00%] [G loss: 3.170596]
epoch:0 step:52 [D loss: 0.209872, acc.: 92.97%] [G loss: 4.259185]
epoch:0 step:53 [D loss: 0.196986, acc.: 90.62%] [G loss: 4.931921]
epoch:0 step:54 [D loss: 0.306104, acc.: 84.38%] [G loss: 5.235296]
epoch:0 step:55 [D loss: 0.334466, acc.: 83.59%] [G loss: 5.417354]
epoch:0 step:56 [D loss: 0.253565, acc.: 91.41%] [G loss: 4.505440]
epoch:0 step:57 [D loss: 0.140960, acc.: 95.31%] [G loss: 4.404570]
epoch:0 step:58 [D loss: 0.307718, acc.: 89.84%] [G loss: 4.663148]
epoch:0 step:59 [D loss: 0.218102, acc.: 90.62%] [G loss: 4.825623]
epoch:0 step:60 [D loss: 0.093966, acc.: 97.66%] [G loss: 4.887771]
epoch:0 step:61 [D loss: 0.049815, acc.: 98.44%] [G loss: 4.508162]
epoch:0 step:62 [D loss: 0.061797, acc.: 99.22%] [G loss: 4.348686]
epoch:0 step:63 [D loss: 0.065821, acc.: 100.00%] [G loss: 4.944210]
epoch:0 step:64 [D loss: 0.088221, acc.: 99.22

epoch:0 step:170 [D loss: 0.264798, acc.: 89.84%] [G loss: 4.689801]
epoch:0 step:171 [D loss: 0.289368, acc.: 91.41%] [G loss: 4.260958]
epoch:0 step:172 [D loss: 0.403678, acc.: 78.91%] [G loss: 6.003686]
epoch:0 step:173 [D loss: 0.444677, acc.: 81.25%] [G loss: 4.809316]
epoch:0 step:174 [D loss: 1.057758, acc.: 50.00%] [G loss: 2.552339]
epoch:0 step:175 [D loss: 0.388083, acc.: 77.34%] [G loss: 5.243370]
epoch:0 step:176 [D loss: 0.848876, acc.: 58.59%] [G loss: 3.920119]
epoch:0 step:177 [D loss: 0.839895, acc.: 68.75%] [G loss: 6.220595]
epoch:0 step:178 [D loss: 1.259574, acc.: 38.28%] [G loss: 5.254566]
epoch:0 step:179 [D loss: 0.543052, acc.: 78.12%] [G loss: 4.138112]
epoch:0 step:180 [D loss: 0.804387, acc.: 55.47%] [G loss: 2.534793]
epoch:0 step:181 [D loss: 1.264817, acc.: 44.53%] [G loss: 5.989539]
epoch:0 step:182 [D loss: 2.097335, acc.: 19.53%] [G loss: 5.097589]
epoch:0 step:183 [D loss: 2.009154, acc.: 34.38%] [G loss: 4.414429]
epoch:0 step:184 [D loss: 1.271578

epoch:0 step:285 [D loss: 0.682873, acc.: 64.84%] [G loss: 2.108847]
epoch:0 step:286 [D loss: 0.736320, acc.: 65.62%] [G loss: 2.304558]
epoch:0 step:287 [D loss: 0.815538, acc.: 54.69%] [G loss: 2.801808]
epoch:0 step:288 [D loss: 0.713849, acc.: 61.72%] [G loss: 2.691988]
epoch:0 step:289 [D loss: 0.673174, acc.: 70.31%] [G loss: 1.927753]
epoch:0 step:290 [D loss: 0.765440, acc.: 53.12%] [G loss: 1.882991]
epoch:0 step:291 [D loss: 1.078601, acc.: 48.44%] [G loss: 1.980667]
epoch:0 step:292 [D loss: 0.606881, acc.: 61.72%] [G loss: 2.322834]
epoch:0 step:293 [D loss: 0.459010, acc.: 79.69%] [G loss: 2.686099]
epoch:0 step:294 [D loss: 0.734033, acc.: 60.16%] [G loss: 2.320488]
epoch:0 step:295 [D loss: 0.643704, acc.: 63.28%] [G loss: 1.966963]
epoch:0 step:296 [D loss: 0.497332, acc.: 75.78%] [G loss: 2.547417]
epoch:0 step:297 [D loss: 0.782868, acc.: 58.59%] [G loss: 1.675955]
epoch:0 step:298 [D loss: 0.681166, acc.: 64.84%] [G loss: 2.079528]
epoch:0 step:299 [D loss: 0.653427

epoch:0 step:403 [D loss: 0.717385, acc.: 59.38%] [G loss: 1.717747]
epoch:0 step:404 [D loss: 0.746324, acc.: 54.69%] [G loss: 1.436246]
epoch:0 step:405 [D loss: 0.639647, acc.: 61.72%] [G loss: 1.395625]
epoch:0 step:406 [D loss: 0.642713, acc.: 67.19%] [G loss: 1.781107]
epoch:0 step:407 [D loss: 0.689210, acc.: 64.06%] [G loss: 1.578618]
epoch:0 step:408 [D loss: 0.611341, acc.: 68.75%] [G loss: 2.007642]
epoch:0 step:409 [D loss: 0.577578, acc.: 67.19%] [G loss: 1.834393]
epoch:0 step:410 [D loss: 0.703861, acc.: 59.38%] [G loss: 1.726760]
epoch:0 step:411 [D loss: 0.657081, acc.: 63.28%] [G loss: 1.695038]
epoch:0 step:412 [D loss: 0.772018, acc.: 59.38%] [G loss: 1.511638]
epoch:0 step:413 [D loss: 0.961374, acc.: 43.75%] [G loss: 1.612596]
epoch:0 step:414 [D loss: 0.843562, acc.: 49.22%] [G loss: 1.552475]
epoch:0 step:415 [D loss: 0.812754, acc.: 46.88%] [G loss: 1.601803]
epoch:0 step:416 [D loss: 0.619324, acc.: 66.41%] [G loss: 1.748305]
epoch:0 step:417 [D loss: 0.823094

...

epoch:29 step:27577 [D loss: 0.226259, acc.: 92.19%] [G loss: 2.701303]
epoch:29 step:27578 [D loss: 0.039498, acc.: 100.00%] [G loss: 1.354953]
epoch:29 step:27579 [D loss: 0.797829, acc.: 55.47%] [G loss: 3.099097]
epoch:29 step:27580 [D loss: 0.210367, acc.: 91.41%] [G loss: 3.176332]
epoch:29 step:27581 [D loss: 0.106726, acc.: 99.22%] [G loss: 3.158405]
epoch:29 step:27582 [D loss: 0.034484, acc.: 100.00%] [G loss: 3.686164]
epoch:29 step:27583 [D loss: 0.396419, acc.: 78.91%] [G loss: 3.383944]
epoch:29 step:27584 [D loss: 1.072544, acc.: 51.56%] [G loss: 1.837684]
epoch:29 step:27585 [D loss: 0.163402, acc.: 96.88%] [G loss: 1.022491]
epoch:29 step:27586 [D loss: 0.291812, acc.: 84.38%] [G loss: 2.705895]
epoch:29 step:27587 [D loss: 0.054435, acc.: 100.00%] [G loss: 2.895245]
epoch:29 step:27588 [D loss: 0.116972, acc.: 100.00%] [G loss: 1.523711]
epoch:29 step:27589 [D loss: 0.161605, acc.: 98.44%] [G loss: 2.296528]
epoch:29 step:27590 [D loss: 0.071706, acc.: 99.22%] [G loss

epoch:29 step:27690 [D loss: 0.381427, acc.: 82.81%] [G loss: 1.216360]
epoch:29 step:27691 [D loss: 0.555718, acc.: 72.66%] [G loss: 1.550380]
epoch:29 step:27692 [D loss: 0.611526, acc.: 68.75%] [G loss: 1.485785]
epoch:29 step:27693 [D loss: 0.444327, acc.: 85.16%] [G loss: 1.393849]
epoch:29 step:27694 [D loss: 0.597832, acc.: 66.41%] [G loss: 0.984916]
epoch:29 step:27695 [D loss: 0.583227, acc.: 71.88%] [G loss: 1.148434]
epoch:29 step:27696 [D loss: 0.581541, acc.: 69.53%] [G loss: 1.209403]
epoch:29 step:27697 [D loss: 0.273022, acc.: 93.75%] [G loss: 1.195002]
epoch:29 step:27698 [D loss: 0.676652, acc.: 60.16%] [G loss: 1.480438]
epoch:29 step:27699 [D loss: 0.445345, acc.: 87.50%] [G loss: 1.525149]
epoch:29 step:27700 [D loss: 0.390325, acc.: 87.50%] [G loss: 1.571841]
epoch:29 step:27701 [D loss: 0.501242, acc.: 80.47%] [G loss: 1.232576]
epoch:29 step:27702 [D loss: 0.434090, acc.: 84.38%] [G loss: 1.468566]
epoch:29 step:27703 [D loss: 0.399190, acc.: 80.47%] [G loss: 1.

epoch:29 step:27804 [D loss: 0.606045, acc.: 65.62%] [G loss: 0.934630]
epoch:29 step:27805 [D loss: 0.400810, acc.: 78.91%] [G loss: 1.209384]
epoch:29 step:27806 [D loss: 0.189932, acc.: 95.31%] [G loss: 1.689720]
epoch:29 step:27807 [D loss: 0.273032, acc.: 93.75%] [G loss: 1.616303]
epoch:29 step:27808 [D loss: 0.159380, acc.: 99.22%] [G loss: 2.388522]
epoch:29 step:27809 [D loss: 0.791008, acc.: 53.12%] [G loss: 1.562628]
epoch:29 step:27810 [D loss: 0.609621, acc.: 63.28%] [G loss: 1.791037]
epoch:29 step:27811 [D loss: 0.771598, acc.: 49.22%] [G loss: 1.634838]
epoch:29 step:27812 [D loss: 0.792376, acc.: 55.47%] [G loss: 0.462559]
epoch:29 step:27813 [D loss: 0.736904, acc.: 54.69%] [G loss: 1.478487]
epoch:29 step:27814 [D loss: 0.314106, acc.: 82.03%] [G loss: 2.144861]
epoch:29 step:27815 [D loss: 1.047469, acc.: 39.84%] [G loss: 0.945944]
epoch:29 step:27816 [D loss: 0.398343, acc.: 89.84%] [G loss: 1.387189]
epoch:29 step:27817 [D loss: 0.144595, acc.: 96.88%] [G loss: 1.

epoch:29 step:27918 [D loss: 0.850279, acc.: 57.03%] [G loss: 2.028532]
epoch:29 step:27919 [D loss: 0.657866, acc.: 60.16%] [G loss: 1.606085]
epoch:29 step:27920 [D loss: 0.398292, acc.: 84.38%] [G loss: 1.736953]
epoch:29 step:27921 [D loss: 0.173118, acc.: 99.22%] [G loss: 1.571135]
epoch:29 step:27922 [D loss: 0.104684, acc.: 100.00%] [G loss: 2.105829]
epoch:29 step:27923 [D loss: 0.157068, acc.: 96.88%] [G loss: 2.205333]
epoch:29 step:27924 [D loss: 0.510658, acc.: 73.44%] [G loss: 2.255005]
epoch:29 step:27925 [D loss: 0.672283, acc.: 58.59%] [G loss: 1.710766]
epoch:29 step:27926 [D loss: 0.693092, acc.: 60.94%] [G loss: 1.752663]
epoch:29 step:27927 [D loss: 0.896947, acc.: 45.31%] [G loss: 1.748425]
epoch:29 step:27928 [D loss: 0.777373, acc.: 56.25%] [G loss: 1.637267]
epoch:29 step:27929 [D loss: 0.677192, acc.: 61.72%] [G loss: 1.686040]
epoch:29 step:27930 [D loss: 0.653899, acc.: 61.72%] [G loss: 1.434088]
epoch:29 step:27931 [D loss: 0.327205, acc.: 90.62%] [G loss: 1

epoch:29 step:28033 [D loss: 0.313826, acc.: 82.03%] [G loss: 1.862371]
epoch:29 step:28034 [D loss: 1.128102, acc.: 45.31%] [G loss: 1.667502]
epoch:29 step:28035 [D loss: 1.159333, acc.: 31.25%] [G loss: 1.423984]
epoch:29 step:28036 [D loss: 0.301397, acc.: 94.53%] [G loss: 1.545533]
epoch:29 step:28037 [D loss: 0.373043, acc.: 88.28%] [G loss: 1.436424]
epoch:29 step:28038 [D loss: 0.903927, acc.: 50.00%] [G loss: 0.970527]
epoch:29 step:28039 [D loss: 0.619667, acc.: 67.97%] [G loss: 1.753277]
epoch:29 step:28040 [D loss: 0.522087, acc.: 76.56%] [G loss: 1.630170]
epoch:29 step:28041 [D loss: 0.259302, acc.: 90.62%] [G loss: 1.732653]
epoch:29 step:28042 [D loss: 0.525931, acc.: 71.88%] [G loss: 1.625619]
epoch:29 step:28043 [D loss: 0.150655, acc.: 96.09%] [G loss: 1.893654]
epoch:29 step:28044 [D loss: 0.193965, acc.: 97.66%] [G loss: 2.036582]
epoch:29 step:28045 [D loss: 0.652695, acc.: 60.94%] [G loss: 1.372816]
epoch:29 step:28046 [D loss: 0.231975, acc.: 91.41%] [G loss: 1.